# Characterizing Collaborators

In this notebook, we shall characterize collaborators. We shall do that in the following way:

There are three reasons: misconduct, plagiarism, and mistake.

For each academic age group within retracted and matched scientists **at the time of retraction**, we shall conduct three analysis and create three tables:

#### Retained for retracted vs. matched
1. Table 1 comparing the **retained** collaborators of retracted and matched scientists in terms of their (a) mean academic age, (b) average number of papers, (c) average number of citations, (d) average number of collaborators, all **at the time of collaboration**. The table will also contain median, standard deviation, and p-value for t-test.

#### Gained for retracted vs. matched
2. Table 2 comparing the **gained/new** collaborators of retracted and matched scientists in terms of their (a) mean academic age, (b) average number of papers, (c) average number of citations, (d) average number of collaborators, all **at the time of collaboration**. The table will also contain median, standard deviation, and p-value for t-test.

#### Retained vs. lost for retracted vs. matched
3. Table 3 comparing the **retained** collaborators of retracted and matched scientists to those **lost** in terms of their (a) mean academic age, (b) average number of papers, (c) average number of citations, (d) average number of collaborators, all **at the time of retraction**. The table will be produced by difference in differences approach where we shall first compute the averages for each field (papers, citations, etc.) for retained and lost for retracted and matched. Then we shall compute the difference between retained for retracted and matched, and between lost for retracted and matched. Finally we shall take the difference in difference (DiD) i.e. **RETAINED-LOST**. The table will also contain median, standard deviation, and p-value for t-test.



In [1]:
import pandas as pd
import sys
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats

/usr/local/lib/python3.9/site-packages/scipy/__init__.py:138: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.25.2)
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion} is required for this version of "


In [2]:
INDIR = "/Users/sm9654/desktop/NYUAD/nyuad-research/retraction_openalex/retraction_effects_on_academic_careers/data/processed/"
INDIR_MATCHING = INDIR+"/author_matching/"
INDIR_COLLAB = INDIR+"/collaborator_quality_analysis/"

df = pd.read_csv(INDIR_COLLAB+"/1Dcollaborators_for_matched_sample_30.csv")

print(df.shape)

df.head()

(773033, 20)


,MAGAID,MAGCollabAID,RetractionYear,MAGCollaborationYear,ScientistType,CollabGenderizeGender,CollabGenderizeConfidence,CollabMAGFirstPubYear,CollabMAGCumPapersYearAtRetraction,CollabMAGCumPapersAtRetraction,CollabMAGCumCitationsYearAtRetraction,CollabMAGCumCitationsAtRetraction,CollabMAGCumCollaboratorsYearAtRetraction,CollabMAGCumCollaboratorsAtRetraction,CollabMAGCumPapersYearAtCollaboration,CollabMAGCumPapersAtCollaboration,CollabMAGCumCitationsYearAtCollaboration,CollabMAGCumCitationsAtCollaboration,CollabMAGCumCollaboratorsYearAtCollaboration,CollabMAGCumCollaboratorsAtCollaboration
0,2.105038e+09,2004120834,1994.0,1983.0,retracted,male,0.99,1967.0,1994.0,104.0,1994.0,3683.0,1994.0,83.0,1983.0,47.0,1983.0,1076.0,1983.0,46
1,2.105038e+09,2124401064,1994.0,1983.0,retracted,male,0.74,1964.0,1994.0,78.0,1994.0,2668.0,1994.0,81.0,1983.0,47.0,1983.0,1305.0,1983.0,37
2,2.105038e+09,2486043001,1994.0,1983.0,retracted,male,0.60,1971.0,1983.0,10.0,1994.0,532.0,1983.0,18.0,1983.0,10.0,1983.0,199.0,1983.0,18
3,2.105038e+09,2124401064,1994.0,1992.0,retracted,male,0.74,1964.0,1994.0,78.0,1994.0,2668.0,1994.0,81.0,1992.0,74.0,1992.0,2449.0,1992.0,71
4,2.105038e+09,2276877851,1994.0,1992.0,retracted,female,0.98,1984.0,1993.0,16.0,1994.0,136.0,1993.0,31.0,1992.0,14.0,1992.0,90.0,1992.0,27


In [3]:
print(df.shape)

(773033, 20)


In [4]:
df.MAGCollabAID.nunique()

411911

### Preprocessing

In [5]:
# Let us first augment the academic age of MAGAIDs. We will also add other columns to be used later

# Reading files used for matching

df_treatment = pd.read_csv(INDIR_MATCHING+"/RWMAG_rematched_treatment_augmented_rematching_30perc.csv",
                    usecols=['MAGAID','RetractionYear','NumRetentionW5','NumNewCollaboratorsW5','ReasonPropagatedMajorityOfMajority',
                            'MAGAIDFirstORLastAuthorFlag'])\
                    .drop_duplicates()\
                    .rename(columns={
                                    'AcademicAgeBeforeRetraction': 'AcademicAgeAtRetraction'})

df_control = pd.read_csv(INDIR_MATCHING+"/RWMAG_rematched_control_augmented_rematching_30perc.csv",
                    usecols=['MatchMAGAID','RetractionYear','NumRetentionW5','NumNewCollaboratorsW5','ReasonPropagatedMajorityOfMajority',
                            'MAGAIDFirstORLastAuthorFlag'])\
                    .drop_duplicates()\
                    .rename(columns={'MatchMAGAID':'MAGAID'})

# Filtering process for choosing only first and last authors

df_treatment = df_treatment[df_treatment['MAGAIDFirstORLastAuthorFlag']=='MAGFirstOrLastAuthor']

df_control = df_control[df_control['MAGAIDFirstORLastAuthorFlag']=='MAGFirstOrLastAuthor']


df_treatment_control = pd.concat([df_treatment,df_control])

# Filtering process
df = df[df['MAGAID'].isin(df_treatment_control['MAGAID'])]

# Removing reasons that are not misconduct, plagiarism, mistake

df_treatment_control = df_treatment_control[df_treatment_control.\
                                        ReasonPropagatedMajorityOfMajority.isin(['misconduct',
                                                                                'plagiarism',
                                                                                'mistake'])]

df_treatment_control

,MAGAID,RetractionYear,ReasonPropagatedMajorityOfMajority,MAGAIDFirstORLastAuthorFlag,NumRetentionW5,NumNewCollaboratorsW5
20,1.839367e+08,2007.0,mistake,MAGFirstOrLastAuthor,8,79
22,2.004364e+08,2012.0,misconduct,MAGFirstOrLastAuthor,13,18
23,2.066031e+08,2012.0,mistake,MAGFirstOrLastAuthor,12,28
25,2.072804e+08,2012.0,misconduct,MAGFirstOrLastAuthor,20,88
26,2.074934e+08,2008.0,mistake,MAGFirstOrLastAuthor,13,55
...,...,...,...,...,...,...
5409,2.135624e+09,2014.0,plagiarism,MAGFirstOrLastAuthor,1,2
5411,2.127710e+09,2015.0,plagiarism,MAGFirstOrLastAuthor,3,25
5412,1.974243e+09,2013.0,mistake,MAGFirstOrLastAuthor,3,2
5416,2.077873e+09,2008.0,misconduct,MAGFirstOrLastAuthor,2,3


In [6]:
# Merging that with df

df2 = df.merge(df_treatment_control.drop(columns=['NumRetentionW5','NumNewCollaboratorsW5']), 
                                         on=['MAGAID','RetractionYear'])
df2

,MAGAID,MAGCollabAID,RetractionYear,MAGCollaborationYear,ScientistType,CollabGenderizeGender,CollabGenderizeConfidence,CollabMAGFirstPubYear,CollabMAGCumPapersYearAtRetraction,CollabMAGCumPapersAtRetraction,...,CollabMAGCumCollaboratorsYearAtRetraction,CollabMAGCumCollaboratorsAtRetraction,CollabMAGCumPapersYearAtCollaboration,CollabMAGCumPapersAtCollaboration,CollabMAGCumCitationsYearAtCollaboration,CollabMAGCumCitationsAtCollaboration,CollabMAGCumCollaboratorsYearAtCollaboration,CollabMAGCumCollaboratorsAtCollaboration,ReasonPropagatedMajorityOfMajority,MAGAIDFirstORLastAuthorFlag
0,2.033335e+09,1917877966,1995.0,1999.0,retracted,male,0.99,1994.0,1994.0,1.0,...,1994.0,2.0,1999.0,14.0,1999.0,99.0,1999.0,31,mistake,MAGFirstOrLastAuthor
1,2.033335e+09,2169118091,1995.0,1999.0,retracted,male,1.00,1995.0,1995.0,1.0,...,1995.0,3.0,1999.0,9.0,1999.0,8.0,1999.0,23,mistake,MAGFirstOrLastAuthor
2,2.033335e+09,275085591,1995.0,1997.0,retracted,male,0.99,1994.0,1995.0,5.0,...,1995.0,11.0,1997.0,12.0,1997.0,53.0,1997.0,22,mistake,MAGFirstOrLastAuthor
3,2.033335e+09,2111014462,1995.0,1997.0,retracted,female,0.98,1988.0,1995.0,17.0,...,1995.0,30.0,1997.0,22.0,1997.0,921.0,1997.0,42,mistake,MAGFirstOrLastAuthor
4,2.033335e+09,2622920657,1995.0,1997.0,retracted,male,0.99,1991.0,1995.0,17.0,...,1995.0,33.0,1997.0,26.0,1997.0,306.0,1997.0,44,mistake,MAGFirstOrLastAuthor
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195007,2.147516e+09,2144146330,2009.0,2016.0,matched,male,0.81,1982.0,2009.0,284.0,...,2009.0,464.0,2016.0,563.0,2016.0,16846.0,2016.0,3686,mistake,MAGFirstOrLastAuthor
195008,2.147516e+09,3146371485,2009.0,2015.0,matched,male,0.60,2015.0,NaN,0.0,...,NaN,0.0,2015.0,1.0,2015.0,1.0,2015.0,4,mistake,MAGFirstOrLastAuthor
195009,2.147516e+09,3149630639,2009.0,2014.0,matched,male,0.97,2014.0,NaN,0.0,...,NaN,0.0,2014.0,1.0,NaN,0.0,2014.0,4,mistake,MAGFirstOrLastAuthor
195010,2.147516e+09,3165764306,2009.0,2014.0,matched,male,0.81,2014.0,NaN,0.0,...,NaN,0.0,2014.0,1.0,NaN,0.0,2014.0,4,mistake,MAGFirstOrLastAuthor


In [7]:
# Let us first compute academic age at retraction and at collaboration for collaborators
df2['CollabAcademicAgeAtRetraction'] = df2['RetractionYear']-df2['CollabMAGFirstPubYear']

df2['CollabAcademicAgeAtCollaboration'] = df2['MAGCollaborationYear']-df2['CollabMAGFirstPubYear']

# So negatives are possible in academic age at retraction but not collaboration
df2.CollabAcademicAgeAtRetraction.describe()

count    195012.000000
mean          9.226478
std          13.767529
min         -30.000000
25%           0.000000
50%           7.000000
75%          17.000000
max         215.000000
Name: CollabAcademicAgeAtRetraction, dtype: float64

In [8]:
# Let us first identify if the collaboration was pre- or post-retraction

def get_prepost_flag(row):
    if(row['MAGCollaborationYear'] <= row['RetractionYear']):
        return 'pre'
    else:
        if((row['MAGCollaborationYear']-row['RetractionYear'])<=5):
            return 'post5'
        else:
            return 'post'

df2['PrePostFlag5'] = df2.apply(lambda row: get_prepost_flag(row), axis=1)

In [9]:
# Let us remove the collaborators that are "post"

df3 = df2[~df2.PrePostFlag5.eq('post')]


In [10]:
df3.head()

,MAGAID,MAGCollabAID,RetractionYear,MAGCollaborationYear,ScientistType,CollabGenderizeGender,CollabGenderizeConfidence,CollabMAGFirstPubYear,CollabMAGCumPapersYearAtRetraction,CollabMAGCumPapersAtRetraction,...,CollabMAGCumPapersAtCollaboration,CollabMAGCumCitationsYearAtCollaboration,CollabMAGCumCitationsAtCollaboration,CollabMAGCumCollaboratorsYearAtCollaboration,CollabMAGCumCollaboratorsAtCollaboration,ReasonPropagatedMajorityOfMajority,MAGAIDFirstORLastAuthorFlag,CollabAcademicAgeAtRetraction,CollabAcademicAgeAtCollaboration,PrePostFlag5
0,2.033335e+09,1917877966,1995.0,1999.0,retracted,male,0.99,1994.0,1994.0,1.0,...,14.0,1999.0,99.0,1999.0,31,mistake,MAGFirstOrLastAuthor,1.0,5.0,post5
1,2.033335e+09,2169118091,1995.0,1999.0,retracted,male,1.00,1995.0,1995.0,1.0,...,9.0,1999.0,8.0,1999.0,23,mistake,MAGFirstOrLastAuthor,0.0,4.0,post5
2,2.033335e+09,275085591,1995.0,1997.0,retracted,male,0.99,1994.0,1995.0,5.0,...,12.0,1997.0,53.0,1997.0,22,mistake,MAGFirstOrLastAuthor,1.0,3.0,post5
3,2.033335e+09,2111014462,1995.0,1997.0,retracted,female,0.98,1988.0,1995.0,17.0,...,22.0,1997.0,921.0,1997.0,42,mistake,MAGFirstOrLastAuthor,7.0,9.0,post5
4,2.033335e+09,2622920657,1995.0,1997.0,retracted,male,0.99,1991.0,1995.0,17.0,...,26.0,1997.0,306.0,1997.0,44,mistake,MAGFirstOrLastAuthor,4.0,6.0,post5


In [11]:
df3.columns

Index(['MAGAID', 'MAGCollabAID', 'RetractionYear', 'MAGCollaborationYear',
       'ScientistType', 'CollabGenderizeGender', 'CollabGenderizeConfidence',
       'CollabMAGFirstPubYear', 'CollabMAGCumPapersYearAtRetraction',
       'CollabMAGCumPapersAtRetraction',
       'CollabMAGCumCitationsYearAtRetraction',
       'CollabMAGCumCitationsAtRetraction',
       'CollabMAGCumCollaboratorsYearAtRetraction',
       'CollabMAGCumCollaboratorsAtRetraction',
       'CollabMAGCumPapersYearAtCollaboration',
       'CollabMAGCumPapersAtCollaboration',
       'CollabMAGCumCitationsYearAtCollaboration',
       'CollabMAGCumCitationsAtCollaboration',
       'CollabMAGCumCollaboratorsYearAtCollaboration',
       'CollabMAGCumCollaboratorsAtCollaboration',
       'ReasonPropagatedMajorityOfMajority', 'MAGAIDFirstORLastAuthorFlag',
       'CollabAcademicAgeAtRetraction', 'CollabAcademicAgeAtCollaboration',
       'PrePostFlag5'],
      dtype='object')

In [12]:
# For each MAGAID, let us create a set of collaborators pre- and post- retraction

df4 = df3.groupby(['MAGAID','RetractionYear','PrePostFlag5'])\
                        ['MAGCollabAID'].apply(set).unstack().reset_index()


# Converting pre- and post5 columns to set so we can do set operations
df4['pre'] = df4['pre'].apply(lambda d: d if isinstance(d, set) else set())
df4['post5'] = df4['post5'].apply(lambda d: d if isinstance(d, set) else set())


# COLLABORATOR RETENTION

# Computing number of collaborators retained
df4['NumRetentionW5'] = df4.apply(lambda row: len(row.post5.intersection(row.pre)), 
                            axis=1)

# Creating the list of collaborators retained
df4['CollabAIDRetainedW5'] = df4.apply(lambda row: row.post5.intersection(row.pre), 
                                                    axis=1)


# Creating list of collaborators lost
df4['CollabAIDLostW5'] = df4.apply(lambda row: row['pre'] - row['CollabAIDRetainedW5'], 
                                                    axis=1)


# COLLABORATOR GAIN

# Computing number of collabortors gained
df4['NumNewCollaboratorsW5'] = df4.apply(lambda row: len(row['post5']-row['pre']), 
                                                    axis=1)

# Creating set of collaborators gained
df4['CollabAIDGainedW5'] = df4.apply(lambda row: row['post5']-row['pre'], 
                                                    axis=1)


df4.head()

PrePostFlag5,MAGAID,RetractionYear,post5,pre,NumRetentionW5,CollabAIDRetainedW5,CollabAIDLostW5,NumNewCollaboratorsW5,CollabAIDGainedW5
0,19100288.0,2002.0,"{18011520, 2181345027, 2608247304, 2043645593,...","{2437904394, 2171993227, 2043645593, 220887567...",5,"{2128982626, 1863203661, 2043645593, 410625722...","{2059887737, 2186312265, 2437904394, 217199322...",23,"{18011520, 2181345027, 2608247304, 2111905563,..."
1,21686935.0,2008.0,"{2646743714, 2800470565, 2117660071, 269971592...","{2170694433, 2064283617, 2327654243, 247583094...",0,{},"{2330648593, 2965357269, 3081263517, 250572015...",12,"{2646743714, 2800470565, 2117660071, 269971592..."
2,29680017.0,1997.0,"{2309217606, 2070053831, 1588714087, 252097620...","{287126053, 1971237384, 2023717609, 2273398858...",1,{2920459384},"{1971237384, 2273398858, 1998215821, 230597927...",4,"{2520976201, 1588714087, 2309217606, 2070053831}"
3,33433812.0,2009.0,"{2653850944, 2127386819, 2128942892, 2113925965}","{2653850944, 2127386819, 2572203943, 212894289...",4,"{2653850944, 2127386819, 2128942892, 2113925965}","{2137538457, 2149027090, 2572203943}",0,{}
4,41957466.0,2015.0,"{2034626565, 1848517639, 2597654027, 203201076...","{2152585733, 2142553607, 2110808072, 294896999...",32,"{2047324288, 2133711754, 1719173006, 219254094...","{2152585733, 2142553607, 2110808072, 294896999...",83,"{2034626565, 1848517639, 2597654027, 203201076..."


### Validation of the number of collaborators retained and gained 

We shall validate if the numbers we calculated now match the ones on which matching was done.

In [13]:
# Merging
dfvalidation = df4[['MAGAID','RetractionYear','NumRetentionW5','NumNewCollaboratorsW5']].drop_duplicates().\
                    merge(df_treatment_control, on=['MAGAID','RetractionYear'])

dfvalidation

,MAGAID,RetractionYear,NumRetentionW5_x,NumNewCollaboratorsW5_x,ReasonPropagatedMajorityOfMajority,MAGAIDFirstORLastAuthorFlag,NumRetentionW5_y,NumNewCollaboratorsW5_y
0,1.910029e+07,2002.0,5,23,mistake,MAGFirstOrLastAuthor,5,23
1,2.168694e+07,2008.0,0,12,plagiarism,MAGFirstOrLastAuthor,0,12
2,2.968002e+07,1997.0,1,4,mistake,MAGFirstOrLastAuthor,1,4
3,3.343381e+07,2009.0,4,0,mistake,MAGFirstOrLastAuthor,4,0
4,4.195747e+07,2015.0,32,83,mistake,MAGFirstOrLastAuthor,32,83
...,...,...,...,...,...,...,...,...
1414,3.170688e+09,2012.0,1,4,plagiarism,MAGFirstOrLastAuthor,1,4
1415,3.173544e+09,2011.0,0,2,misconduct,MAGFirstOrLastAuthor,0,2
1416,3.174124e+09,2004.0,1,6,mistake,MAGFirstOrLastAuthor,1,6
1417,3.175436e+09,2015.0,1,14,plagiarism,MAGFirstOrLastAuthor,1,14


In [14]:
# Finally validating

dfvalidation[(dfvalidation.NumRetentionW5_x == dfvalidation.NumRetentionW5_y) & 
            (dfvalidation.NumNewCollaboratorsW5_x == dfvalidation.NumNewCollaboratorsW5_y)]

,MAGAID,RetractionYear,NumRetentionW5_x,NumNewCollaboratorsW5_x,ReasonPropagatedMajorityOfMajority,MAGAIDFirstORLastAuthorFlag,NumRetentionW5_y,NumNewCollaboratorsW5_y
0,1.910029e+07,2002.0,5,23,mistake,MAGFirstOrLastAuthor,5,23
1,2.168694e+07,2008.0,0,12,plagiarism,MAGFirstOrLastAuthor,0,12
2,2.968002e+07,1997.0,1,4,mistake,MAGFirstOrLastAuthor,1,4
3,3.343381e+07,2009.0,4,0,mistake,MAGFirstOrLastAuthor,4,0
4,4.195747e+07,2015.0,32,83,mistake,MAGFirstOrLastAuthor,32,83
...,...,...,...,...,...,...,...,...
1414,3.170688e+09,2012.0,1,4,plagiarism,MAGFirstOrLastAuthor,1,4
1415,3.173544e+09,2011.0,0,2,misconduct,MAGFirstOrLastAuthor,0,2
1416,3.174124e+09,2004.0,1,6,mistake,MAGFirstOrLastAuthor,1,6
1417,3.175436e+09,2015.0,1,14,plagiarism,MAGFirstOrLastAuthor,1,14


**Hence all of them are validated.**

## Analysis

In [15]:
# Our main dataframes are df3 and df4
# Let us look at them first
print(df3.shape)
df3.head()

(143648, 25)


,MAGAID,MAGCollabAID,RetractionYear,MAGCollaborationYear,ScientistType,CollabGenderizeGender,CollabGenderizeConfidence,CollabMAGFirstPubYear,CollabMAGCumPapersYearAtRetraction,CollabMAGCumPapersAtRetraction,...,CollabMAGCumPapersAtCollaboration,CollabMAGCumCitationsYearAtCollaboration,CollabMAGCumCitationsAtCollaboration,CollabMAGCumCollaboratorsYearAtCollaboration,CollabMAGCumCollaboratorsAtCollaboration,ReasonPropagatedMajorityOfMajority,MAGAIDFirstORLastAuthorFlag,CollabAcademicAgeAtRetraction,CollabAcademicAgeAtCollaboration,PrePostFlag5
0,2.033335e+09,1917877966,1995.0,1999.0,retracted,male,0.99,1994.0,1994.0,1.0,...,14.0,1999.0,99.0,1999.0,31,mistake,MAGFirstOrLastAuthor,1.0,5.0,post5
1,2.033335e+09,2169118091,1995.0,1999.0,retracted,male,1.00,1995.0,1995.0,1.0,...,9.0,1999.0,8.0,1999.0,23,mistake,MAGFirstOrLastAuthor,0.0,4.0,post5
2,2.033335e+09,275085591,1995.0,1997.0,retracted,male,0.99,1994.0,1995.0,5.0,...,12.0,1997.0,53.0,1997.0,22,mistake,MAGFirstOrLastAuthor,1.0,3.0,post5
3,2.033335e+09,2111014462,1995.0,1997.0,retracted,female,0.98,1988.0,1995.0,17.0,...,22.0,1997.0,921.0,1997.0,42,mistake,MAGFirstOrLastAuthor,7.0,9.0,post5
4,2.033335e+09,2622920657,1995.0,1997.0,retracted,male,0.99,1991.0,1995.0,17.0,...,26.0,1997.0,306.0,1997.0,44,mistake,MAGFirstOrLastAuthor,4.0,6.0,post5


In [16]:
df4

PrePostFlag5,MAGAID,RetractionYear,post5,pre,NumRetentionW5,CollabAIDRetainedW5,CollabAIDLostW5,NumNewCollaboratorsW5,CollabAIDGainedW5
0,1.910029e+07,2002.0,"{18011520, 2181345027, 2608247304, 2043645593,...","{2437904394, 2171993227, 2043645593, 220887567...",5,"{2128982626, 1863203661, 2043645593, 410625722...","{2059887737, 2186312265, 2437904394, 217199322...",23,"{18011520, 2181345027, 2608247304, 2111905563,..."
1,2.168694e+07,2008.0,"{2646743714, 2800470565, 2117660071, 269971592...","{2170694433, 2064283617, 2327654243, 247583094...",0,{},"{2330648593, 2965357269, 3081263517, 250572015...",12,"{2646743714, 2800470565, 2117660071, 269971592..."
2,2.968002e+07,1997.0,"{2309217606, 2070053831, 1588714087, 252097620...","{287126053, 1971237384, 2023717609, 2273398858...",1,{2920459384},"{1971237384, 2273398858, 1998215821, 230597927...",4,"{2520976201, 1588714087, 2309217606, 2070053831}"
3,3.343381e+07,2009.0,"{2653850944, 2127386819, 2128942892, 2113925965}","{2653850944, 2127386819, 2572203943, 212894289...",4,"{2653850944, 2127386819, 2128942892, 2113925965}","{2137538457, 2149027090, 2572203943}",0,{}
4,4.195747e+07,2015.0,"{2034626565, 1848517639, 2597654027, 203201076...","{2152585733, 2142553607, 2110808072, 294896999...",32,"{2047324288, 2133711754, 1719173006, 219254094...","{2152585733, 2142553607, 2110808072, 294896999...",83,"{2034626565, 1848517639, 2597654027, 203201076..."
...,...,...,...,...,...,...,...,...,...
1414,3.170688e+09,2012.0,"{2152436290, 2122621478, 2798995243, 216039336...","{2798928472, 2531157211, 2901115159}",1,{2531157211},"{2798928472, 2901115159}",4,"{2152436290, 2798995243, 2122621478, 2160393366}"
1415,3.173544e+09,2011.0,"{3120802076, 2236143686}","{2096029443, 2160112133, 2158121610, 230907623...",0,{},"{2096029443, 2160112133, 2158121610, 230907623...",2,"{3120802076, 2236143686}"
1416,3.174124e+09,2004.0,"{2706053123, 2250669861, 2568589385, 222924385...","{2939265617, 2687883010, 2424699715, 2100866894}",1,{2100866894},"{2939265617, 2687883010, 2424699715}",6,"{2706053123, 2250669861, 2568589385, 222924385..."
1417,3.175436e+09,2015.0,"{1805786912, 2999619457, 2658197410, 257933920...","{2130470407, 2395301650, 1455333013, 231293572...",1,{2121913688},"{2240552385, 2130470407, 2333910471, 252033530...",14,"{1805786912, 2999619457, 2658197410, 196858720..."


In [17]:
# Let us first merge df3 and df4

df_A = df3.merge(df4, on=['MAGAID','RetractionYear'])

# Let us also create three flags checking whether current collaborator is retained, gained, or lost

df_A['CollabAIDinRetained'] = df_A.apply(lambda row: row['MAGCollabAID'] in row['CollabAIDRetainedW5'], 
                                          axis=1)

df_A['CollabAIDinGained'] = df_A.apply(lambda row: row['MAGCollabAID'] in row['CollabAIDGainedW5'], 
                                          axis=1)

df_A['CollabAIDinLost'] = df_A.apply(lambda row: row['MAGCollabAID'] in row['CollabAIDLostW5'], 
                                          axis=1)

df_A.head(3)


,MAGAID,MAGCollabAID,RetractionYear,MAGCollaborationYear,ScientistType,CollabGenderizeGender,CollabGenderizeConfidence,CollabMAGFirstPubYear,CollabMAGCumPapersYearAtRetraction,CollabMAGCumPapersAtRetraction,...,post5,pre,NumRetentionW5,CollabAIDRetainedW5,CollabAIDLostW5,NumNewCollaboratorsW5,CollabAIDGainedW5,CollabAIDinRetained,CollabAIDinGained,CollabAIDinLost
0,2.033335e+09,1917877966,1995.0,1999.0,retracted,male,0.99,1994.0,1994.0,1.0,...,"{1973921793, 2041262087, 2570043915, 213721346...","{2041262087, 2590561305, 1940968476, 211428099...",23,"{2690057605, 2041262087, 2466141063, 230478619...","{2590561305, 1940968476, 2657016352, 265692164...",60,"{1973921793, 2308182914, 1147890307, 265602649...",False,True,False
1,2.033335e+09,2169118091,1995.0,1999.0,retracted,male,1.00,1995.0,1995.0,1.0,...,"{1973921793, 2041262087, 2570043915, 213721346...","{2041262087, 2590561305, 1940968476, 211428099...",23,"{2690057605, 2041262087, 2466141063, 230478619...","{2590561305, 1940968476, 2657016352, 265692164...",60,"{1973921793, 2308182914, 1147890307, 265602649...",False,True,False
2,2.033335e+09,275085591,1995.0,1997.0,retracted,male,0.99,1994.0,1995.0,5.0,...,"{1973921793, 2041262087, 2570043915, 213721346...","{2041262087, 2590561305, 1940968476, 211428099...",23,"{2690057605, 2041262087, 2466141063, 230478619...","{2590561305, 1940968476, 2657016352, 265692164...",60,"{1973921793, 2308182914, 1147890307, 265602649...",True,False,False


In [18]:
# Sensibility checks

df_A[['CollabAIDinRetained','CollabAIDinGained','CollabAIDinLost']].value_counts()

CollabAIDinRetained  CollabAIDinGained  CollabAIDinLost
False                False              True               57073
                     True               False              50210
True                 False              False              36365
Name: count, dtype: int64

In [19]:
df_A.columns, df_A.shape

(Index(['MAGAID', 'MAGCollabAID', 'RetractionYear', 'MAGCollaborationYear',
        'ScientistType', 'CollabGenderizeGender', 'CollabGenderizeConfidence',
        'CollabMAGFirstPubYear', 'CollabMAGCumPapersYearAtRetraction',
        'CollabMAGCumPapersAtRetraction',
        'CollabMAGCumCitationsYearAtRetraction',
        'CollabMAGCumCitationsAtRetraction',
        'CollabMAGCumCollaboratorsYearAtRetraction',
        'CollabMAGCumCollaboratorsAtRetraction',
        'CollabMAGCumPapersYearAtCollaboration',
        'CollabMAGCumPapersAtCollaboration',
        'CollabMAGCumCitationsYearAtCollaboration',
        'CollabMAGCumCitationsAtCollaboration',
        'CollabMAGCumCollaboratorsYearAtCollaboration',
        'CollabMAGCumCollaboratorsAtCollaboration',
        'ReasonPropagatedMajorityOfMajority', 'MAGAIDFirstORLastAuthorFlag',
        'CollabAcademicAgeAtRetraction', 'CollabAcademicAgeAtCollaboration',
        'PrePostFlag5', 'post5', 'pre', 'NumRetentionW5', 'CollabAIDRetainedW5',

## DANGER ZONE!

This code removes collaborators that have academic age > 70 at the time of collaboration. 

In [20]:
df_A[df_A.CollabAcademicAgeAtCollaboration.gt(70) & df_A.ScientistType.eq('retracted')].MAGAID.nunique()

81

In [21]:
df_A = df_A[df_A.CollabAcademicAgeAtCollaboration.le(70)]

### A1: Collaborators retained: retracted vs. matched

In [22]:
#Let us now modify df_A1 such that we remove all rows with collaborations pre-retraction

df_A1_post = df_A[df_A['PrePostFlag5']=='post5']

In [23]:
# Now we shall groupby MAGAID, MAGCollabAID, RetractionYear, and sort by MAGCollaborationYear
# Then I shall extract the earliest collaboration year post retraction

df_A1_firstcollabs = df_A1_post.groupby(['MAGAID','MAGCollabAID','RetractionYear'])['MAGCollaborationYear']\
                        .min().reset_index()\
                        .rename(columns={'MAGCollaborationYear':'FirstPostRetractionMAGCollaborationYear'})


# Now we shall merge the new column with A1

df_A1_w_firstcollabs = df_A1_post.merge(df_A1_firstcollabs,
                                   on=['MAGAID','MAGCollabAID','RetractionYear'])

df_A1_w_firstcollabs.shape

(66141, 36)

In [24]:
# Sensibility checks

df_A1_w_firstcollabs.sort_values(by=['MAGAID','MAGCollabAID','MAGCollaborationYear'])\
            [['MAGAID','MAGCollabAID','MAGCollaborationYear','FirstPostRetractionMAGCollaborationYear']].head(30)

,MAGAID,MAGCollabAID,MAGCollaborationYear,FirstPostRetractionMAGCollaborationYear
2121,19100288.0,18011520,2004.0,2004.0
2128,19100288.0,121410733,2004.0,2004.0
2109,19100288.0,410625722,2005.0,2005.0
2122,19100288.0,1235268530,2004.0,2004.0
2131,19100288.0,1340583028,2006.0,2006.0
2103,19100288.0,1793107545,2003.0,2003.0
2104,19100288.0,1793107545,2004.0,2003.0
2123,19100288.0,1859744180,2004.0,2004.0
2105,19100288.0,1863203661,2003.0,2003.0
2106,19100288.0,1863203661,2004.0,2003.0


In [25]:
# Now let us only extract rows where collaboration year is the first collaboration year

df_A1_w_firstcollabs_only = df_A1_w_firstcollabs[df_A1_w_firstcollabs.MAGCollaborationYear == \
                                                df_A1_w_firstcollabs.FirstPostRetractionMAGCollaborationYear]

df_A1_w_firstcollabs_only.shape

(50527, 36)

In [26]:
df_A1_w_firstcollabs_only.columns

Index(['MAGAID', 'MAGCollabAID', 'RetractionYear', 'MAGCollaborationYear',
       'ScientistType', 'CollabGenderizeGender', 'CollabGenderizeConfidence',
       'CollabMAGFirstPubYear', 'CollabMAGCumPapersYearAtRetraction',
       'CollabMAGCumPapersAtRetraction',
       'CollabMAGCumCitationsYearAtRetraction',
       'CollabMAGCumCitationsAtRetraction',
       'CollabMAGCumCollaboratorsYearAtRetraction',
       'CollabMAGCumCollaboratorsAtRetraction',
       'CollabMAGCumPapersYearAtCollaboration',
       'CollabMAGCumPapersAtCollaboration',
       'CollabMAGCumCitationsYearAtCollaboration',
       'CollabMAGCumCitationsAtCollaboration',
       'CollabMAGCumCollaboratorsYearAtCollaboration',
       'CollabMAGCumCollaboratorsAtCollaboration',
       'ReasonPropagatedMajorityOfMajority', 'MAGAIDFirstORLastAuthorFlag',
       'CollabAcademicAgeAtRetraction', 'CollabAcademicAgeAtCollaboration',
       'PrePostFlag5', 'post5', 'pre', 'NumRetentionW5', 'CollabAIDRetainedW5',
       'CollabAI

In [27]:
def create_stratified_dfs_retention(dfi):
    
    # This function will create 6 dataframes relevant for conducting our analysis
    # 3 of those dataframes will be for relevant columns for treatment
    # rest 3 will be average control. 
    # These will be stratified by treatment and control, and further stratified by seniority
    df_ids = pd.read_csv(INDIR_MATCHING+"/RWMAG_rematched_control_augmented_rematching_30perc.csv",
                    usecols=['MAGAID','MatchMAGAID', 'RetractionYear']).drop_duplicates()
    
    rel_cols = ['MAGAID', 'ScientistType','MAGCollabAID', 'RetractionYear', 'MAGCollaborationYear',
               'CollabMAGCumPapersAtCollaboration', 'CollabMAGCumCitationsAtCollaboration',
               'CollabMAGCumCollaboratorsAtCollaboration', 'ReasonPropagatedMajorityOfMajority',
               'CollabAcademicAgeAtCollaboration', 'CollabAIDinRetained', 'NumRetentionW5']
    
    # Only extracting relevant cols
    dfi = dfi[rel_cols].drop_duplicates()
    
    # Only extract those collaborators that were retained
    dfi = dfi[dfi['CollabAIDinRetained']]
    
    # Dividing into retracted and matched
    df_retracted = dfi[dfi.ScientistType == 'retracted']
    df_nonretracted = dfi[dfi.ScientistType == 'matched']\
                        .rename(columns={'MAGAID':'MatchMAGAID'})\
                        .merge(df_ids, on=['MatchMAGAID','RetractionYear'])
    
    # We also need to makre sure that the retracted scientists have matches with non zero collaborators
    df_retracted = df_retracted[df_retracted.MAGAID.isin(df_nonretracted.MAGAID.unique())]
    
    # We need to make sure that the matches of those who retained 0 collaborators are removed
    df_nonretracted = df_nonretracted[df_nonretracted.MAGAID.isin(df_retracted.MAGAID.unique())]
    
    # Dividing into seniority for retracted
    df_retracted_junior = df_retracted[df_retracted.ReasonPropagatedMajorityOfMajority=='misconduct']
    df_retracted_midcareer = df_retracted[df_retracted.ReasonPropagatedMajorityOfMajority=='plagiarism']
    df_retracted_senior = df_retracted[df_retracted.ReasonPropagatedMajorityOfMajority=='mistake']
    # and matched
    df_nonretracted_junior = df_nonretracted[df_nonretracted.ReasonPropagatedMajorityOfMajority=='misconduct']
    df_nonretracted_midcareer = df_nonretracted[df_nonretracted.ReasonPropagatedMajorityOfMajority=='plagiarism']
    df_nonretracted_senior = df_nonretracted[df_nonretracted.ReasonPropagatedMajorityOfMajority=='mistake']
    
    return df_retracted_junior, df_retracted_midcareer, df_retracted_senior,df_nonretracted_junior, df_nonretracted_midcareer, df_nonretracted_senior
    

In [28]:
df_rj, df_rm, df_rs, df_nrj, df_nrm, df_nrs = create_stratified_dfs_retention(df_A1_w_firstcollabs_only)

In [29]:
df_rj.MAGAID.nunique(), df_rm.MAGAID.nunique(), df_rs.MAGAID.nunique(), df_nrj.MAGAID.nunique(), df_nrm.MAGAID.nunique(), df_nrs.MAGAID.nunique()

(64, 123, 115, 64, 123, 115)

In [30]:
# Let us extract the mean dataframes and merge them for different age categories

def get_mean_df_retention(dfr, dfnr):
    mean_dfr = dfr.groupby('MAGAID')[['CollabAcademicAgeAtCollaboration',
                      'CollabMAGCumPapersAtCollaboration',
                      'CollabMAGCumCitationsAtCollaboration',
                      'CollabMAGCumCollaboratorsAtCollaboration']].mean()
    
    mean_dfnr = dfnr.groupby(['MAGAID','MatchMAGAID'])[['CollabAcademicAgeAtCollaboration',
                      'CollabMAGCumPapersAtCollaboration',
                      'CollabMAGCumCitationsAtCollaboration',
                      'CollabMAGCumCollaboratorsAtCollaboration']].mean()\
                    .groupby('MAGAID').mean()\
                    .rename(columns={'CollabMAGCumPapersAtCollaboration':'MatchCollabMAGCumPapersAtCollaboration',
                                    'CollabAcademicAgeAtCollaboration':'MatchCollabAcademicAgeAtCollaboration',
                                    'CollabMAGCumCitationsAtCollaboration': 'MatchCollabMAGCumCitationsAtCollaboration',
                                    'CollabMAGCumCollaboratorsAtCollaboration': 'MatchCollabMAGCumCollaboratorsAtCollaboration'})
    
    return mean_dfr, mean_dfnr

def mean_confidence_interval(data, confidence=0.95):
    a = 1.0 * np.array(data)
    n = len(a)
    m, se = np.mean(a), stats.sem(a)
    h = se * stats.t.ppf((1 + confidence) / 2., n-1)
    return m, m-h, m+h

def get_stats(dfr, dfnr, column):
    """
    This code will compute the mean, median, std dev. and p-value (as per welch test), and CIs for
    the given column
    """
    
    mean_r = dfr[column].mean()
    median_r = dfr[column].median()
    std_r = dfr[column].std()
    
    mean_nr = dfnr['Match'+column].mean()
    median_nr = dfnr['Match'+column].median()
    std_nr = dfnr['Match'+column].std()
    
    _, pval = stats.ttest_ind(dfr[column],dfnr['Match'+column], equal_var=False)
    
    lst_delta = (dfr[column]- dfnr['Match'+column]).tolist()
    delta_mean, conf_lower, conf_upper = mean_confidence_interval(lst_delta, confidence=0.95)
    
    return {column+'_retracted_mean':round(mean_r,2), 
            column+'_retracted_median':round(median_r,2), 
            column+'_retracted_std':round(std_r,2), 
            column+'_nonretracted_mean':round(mean_nr,2), 
            column+'_nonretracted_median':round(median_nr,2), 
            column+'_nonretracted_std':round(std_nr,2), 
            column+'_delta_mean':round(delta_mean,2), 
            column+'_pval_welch':round(pval,3), 
            column+'_CI_95lower':round(conf_lower,2), 
            column+'_CI_95upper':round(conf_upper,2)}

In [31]:
# Now let us do the comparison

# Let us first get the mean dataframes

mean_dfrj, mean_dfnrj = get_mean_df_retention(df_rj, df_nrj)
mean_dfrm, mean_dfnrm = get_mean_df_retention(df_rm, df_nrm)
mean_dfrs, mean_dfnrs = get_mean_df_retention(df_rs, df_nrs)

mean_dfnrj

,MatchCollabAcademicAgeAtCollaboration,MatchCollabMAGCumPapersAtCollaboration,MatchCollabMAGCumCitationsAtCollaboration,MatchCollabMAGCumCollaboratorsAtCollaboration
MAGAID,,,,
48740240,11.142857,40.142857,443.714286,126.000000
115663519,7.904762,18.238095,268.428571,56.428571
207280435,13.857143,28.500000,439.642857,61.642857
409654858,16.454545,58.272727,2185.818182,219.545455
1646188048,18.000000,62.666667,403.777778,74.333333
...,...,...,...,...
2890048425,16.714286,197.428571,7190.857143,513.857143
2935411574,12.500000,99.500000,1502.000000,158.500000
2935995536,2.666667,2.666667,4.333333,5.666667


In [32]:
exp_fields = ['CollabAcademicAgeAtCollaboration',
                      'CollabMAGCumPapersAtCollaboration',
                      'CollabMAGCumCitationsAtCollaboration',
                      'CollabMAGCumCollaboratorsAtCollaboration']

# Now we should compute outcome variabels for each of the four experience variables.

lst_dicts_retention = []

for exp_field in exp_fields:
    dicts_retention = {}
    
    dict_stats_j = get_stats(mean_dfrj, mean_dfnrj, exp_field)
    dict_stats_m = get_stats(mean_dfrm, mean_dfnrm, exp_field)
    dict_stats_s = get_stats(mean_dfrs, mean_dfnrs, exp_field)
    
    dicts_retention['Misconduct'] = dict_stats_j
    dicts_retention['Plagiarism'] = dict_stats_m
    dicts_retention['Mistake'] = dict_stats_s
    
    lst_dicts_retention.append(dicts_retention)

In [33]:
pd.DataFrame(lst_dicts_retention[0])

,Misconduct,Plagiarism,Mistake
CollabAcademicAgeAtCollaboration_retracted_mean,12.920,12.610,15.110
CollabAcademicAgeAtCollaboration_retracted_median,13.860,12.500,15.460
CollabAcademicAgeAtCollaboration_retracted_std,7.120,6.970,6.750
CollabAcademicAgeAtCollaboration_nonretracted_mean,14.680,15.410,15.880
CollabAcademicAgeAtCollaboration_nonretracted_median,13.360,13.730,14.640
CollabAcademicAgeAtCollaboration_nonretracted_std,6.800,7.870,7.140
CollabAcademicAgeAtCollaboration_delta_mean,-1.770,-2.800,-0.770
CollabAcademicAgeAtCollaboration_pval_welch,0.154,0.003,0.401
CollabAcademicAgeAtCollaboration_CI_95lower,-3.970,-4.530,-2.290
CollabAcademicAgeAtCollaboration_CI_95upper,0.440,-1.070,0.740


In [34]:
pd.DataFrame(lst_dicts_retention[1])

,Misconduct,Plagiarism,Mistake
CollabMAGCumPapersAtCollaboration_retracted_mean,69.800,63.890,69.930
CollabMAGCumPapersAtCollaboration_retracted_median,46.510,44.000,53.000
CollabMAGCumPapersAtCollaboration_retracted_std,96.540,68.830,72.680
CollabMAGCumPapersAtCollaboration_nonretracted_mean,93.750,66.410,74.760
CollabMAGCumPapersAtCollaboration_nonretracted_median,63.280,57.000,61.670
CollabMAGCumPapersAtCollaboration_nonretracted_std,176.220,59.840,55.390
CollabMAGCumPapersAtCollaboration_delta_mean,-23.940,-2.520,-4.820
CollabMAGCumPapersAtCollaboration_pval_welch,0.343,0.759,0.572
CollabMAGCumPapersAtCollaboration_CI_95lower,-75.210,-18.700,-21.620
CollabMAGCumPapersAtCollaboration_CI_95upper,27.320,13.660,11.980


In [35]:
pd.DataFrame(lst_dicts_retention[2])

,Misconduct,Plagiarism,Mistake
CollabMAGCumCitationsAtCollaboration_retracted_mean,1498.480,1158.960,1487.990
CollabMAGCumCitationsAtCollaboration_retracted_median,532.420,410.500,862.220
CollabMAGCumCitationsAtCollaboration_retracted_std,2443.600,2846.240,1947.440
CollabMAGCumCitationsAtCollaboration_nonretracted_mean,1683.140,1009.080,1568.090
CollabMAGCumCitationsAtCollaboration_nonretracted_median,1037.170,497.780,924.000
CollabMAGCumCitationsAtCollaboration_nonretracted_std,2759.870,1440.510,2243.950
CollabMAGCumCitationsAtCollaboration_delta_mean,-184.670,149.880,-80.110
CollabMAGCumCitationsAtCollaboration_pval_welch,0.689,0.603,0.773
CollabMAGCumCitationsAtCollaboration_CI_95lower,-1116.730,-390.570,-609.850
CollabMAGCumCitationsAtCollaboration_CI_95upper,747.400,690.330,449.640


In [36]:
pd.DataFrame(lst_dicts_retention[3])

,Misconduct,Plagiarism,Mistake
CollabMAGCumCollaboratorsAtCollaboration_retracted_mean,194.340,140.040,175.55
CollabMAGCumCollaboratorsAtCollaboration_retracted_median,102.000,70.860,101.00
CollabMAGCumCollaboratorsAtCollaboration_retracted_std,326.470,175.550,203.20
CollabMAGCumCollaboratorsAtCollaboration_nonretracted_mean,272.990,140.570,162.51
CollabMAGCumCollaboratorsAtCollaboration_nonretracted_median,142.930,95.250,120.29
CollabMAGCumCollaboratorsAtCollaboration_nonretracted_std,712.280,170.370,149.90
CollabMAGCumCollaboratorsAtCollaboration_delta_mean,-78.650,-0.530,13.05
CollabMAGCumCollaboratorsAtCollaboration_pval_welch,0.424,0.981,0.58
CollabMAGCumCollaboratorsAtCollaboration_CI_95lower,-281.120,-43.020,-31.83
CollabMAGCumCollaboratorsAtCollaboration_CI_95upper,123.820,41.950,57.92


### A2: Collaborators gained: retracted vs. matched

In [37]:
#Let us now modify df_A1 such that we remove all rows with collaborations pre-retraction

df_A2_post = df_A[df_A['PrePostFlag5']=='post5']

In [38]:
# Now we shall groupby MAGAID, MAGCollabAID, RetractionYear, and sort by MAGCollaborationYear
# Then I shall extract the earliest collaboration year post retraction

df_A2_firstcollabs = df_A2_post.groupby(['MAGAID','MAGCollabAID','RetractionYear'])['MAGCollaborationYear']\
                        .min().reset_index()\
                        .rename(columns={'MAGCollaborationYear':'FirstPostRetractionMAGCollaborationYear'})


# Now we shall merge the new column with A1

df_A2_w_firstcollabs = df_A2_post.merge(df_A2_firstcollabs,
                                   on=['MAGAID','MAGCollabAID','RetractionYear'])

df_A2_w_firstcollabs.shape

(66141, 36)

In [39]:
# Sensibility checks

df_A2_w_firstcollabs.sort_values(by=['MAGAID','MAGCollabAID','MAGCollaborationYear'])\
            [['MAGAID','MAGCollabAID','MAGCollaborationYear','FirstPostRetractionMAGCollaborationYear']].head(30)

,MAGAID,MAGCollabAID,MAGCollaborationYear,FirstPostRetractionMAGCollaborationYear
2121,19100288.0,18011520,2004.0,2004.0
2128,19100288.0,121410733,2004.0,2004.0
2109,19100288.0,410625722,2005.0,2005.0
2122,19100288.0,1235268530,2004.0,2004.0
2131,19100288.0,1340583028,2006.0,2006.0
2103,19100288.0,1793107545,2003.0,2003.0
2104,19100288.0,1793107545,2004.0,2003.0
2123,19100288.0,1859744180,2004.0,2004.0
2105,19100288.0,1863203661,2003.0,2003.0
2106,19100288.0,1863203661,2004.0,2003.0


In [40]:
# Now let us only extract rows where collaboration year is the first collaboration year

df_A2_w_firstcollabs_only = df_A2_w_firstcollabs[df_A2_w_firstcollabs.MAGCollaborationYear == \
                                                df_A2_w_firstcollabs.FirstPostRetractionMAGCollaborationYear]

df_A2_w_firstcollabs_only.shape

(50527, 36)

In [41]:
df_A2_w_firstcollabs_only.columns

Index(['MAGAID', 'MAGCollabAID', 'RetractionYear', 'MAGCollaborationYear',
       'ScientistType', 'CollabGenderizeGender', 'CollabGenderizeConfidence',
       'CollabMAGFirstPubYear', 'CollabMAGCumPapersYearAtRetraction',
       'CollabMAGCumPapersAtRetraction',
       'CollabMAGCumCitationsYearAtRetraction',
       'CollabMAGCumCitationsAtRetraction',
       'CollabMAGCumCollaboratorsYearAtRetraction',
       'CollabMAGCumCollaboratorsAtRetraction',
       'CollabMAGCumPapersYearAtCollaboration',
       'CollabMAGCumPapersAtCollaboration',
       'CollabMAGCumCitationsYearAtCollaboration',
       'CollabMAGCumCitationsAtCollaboration',
       'CollabMAGCumCollaboratorsYearAtCollaboration',
       'CollabMAGCumCollaboratorsAtCollaboration',
       'ReasonPropagatedMajorityOfMajority', 'MAGAIDFirstORLastAuthorFlag',
       'CollabAcademicAgeAtRetraction', 'CollabAcademicAgeAtCollaboration',
       'PrePostFlag5', 'post5', 'pre', 'NumRetentionW5', 'CollabAIDRetainedW5',
       'CollabAI

In [42]:
def create_stratified_dfs_gain(dfi):
    
    # This function will create 6 dataframes relevant for conducting our analysis
    # 3 of those dataframes will be for relevant columns for treatment
    # rest 3 will be average control. 
    # These will be stratified by treatment and control, and further stratified by seniority
    df_ids = pd.read_csv(INDIR_MATCHING+"/RWMAG_rematched_control_augmented_rematching_30perc.csv",
                    usecols=['MAGAID','MatchMAGAID', 'RetractionYear']).drop_duplicates()
    
    rel_cols = ['MAGAID', 'ScientistType','MAGCollabAID', 'RetractionYear', 'MAGCollaborationYear',
               'CollabMAGCumPapersAtCollaboration', 'CollabMAGCumCitationsAtCollaboration',
               'CollabMAGCumCollaboratorsAtCollaboration', 'ReasonPropagatedMajorityOfMajority',
               'CollabAcademicAgeAtCollaboration', 'CollabAIDinGained', 'NumNewCollaboratorsW5']
    
    # Only extracting relevant cols
    dfi = dfi[rel_cols].drop_duplicates()
    
    # Only extract those collaborators that were retained
    dfi = dfi[dfi['CollabAIDinGained']]
    
    # Dividing into retracted and matched
    df_retracted = dfi[dfi.ScientistType == 'retracted']
    df_nonretracted = dfi[dfi.ScientistType == 'matched']\
                        .rename(columns={'MAGAID':'MatchMAGAID'})\
                        .merge(df_ids, on=['MatchMAGAID','RetractionYear'])
    
    # We also need to makre sure that the retracted scientists have matches with non zero collaborators
    df_retracted = df_retracted[df_retracted.MAGAID.isin(df_nonretracted.MAGAID.unique())]
    
    # We need to make sure that the matches of those who retained 0 collaborators are removed
    df_nonretracted = df_nonretracted[df_nonretracted.MAGAID.isin(df_retracted.MAGAID.unique())]
    
    # Dividing into seniority for retracted
    df_retracted_junior = df_retracted[df_retracted.ReasonPropagatedMajorityOfMajority=='misconduct']
    df_retracted_midcareer = df_retracted[df_retracted.ReasonPropagatedMajorityOfMajority=='plagiarism']
    df_retracted_senior = df_retracted[df_retracted.ReasonPropagatedMajorityOfMajority=='mistake']
    # and matched
    df_nonretracted_junior = df_nonretracted[df_nonretracted.ReasonPropagatedMajorityOfMajority=='misconduct']
    df_nonretracted_midcareer = df_nonretracted[df_nonretracted.ReasonPropagatedMajorityOfMajority=='plagiarism']
    df_nonretracted_senior = df_nonretracted[df_nonretracted.ReasonPropagatedMajorityOfMajority=='mistake']
    
    return df_retracted_junior, df_retracted_midcareer, df_retracted_senior,df_nonretracted_junior, df_nonretracted_midcareer, df_nonretracted_senior
    

In [43]:
df_rj, df_rm, df_rs, df_nrj, df_nrm, df_nrs = create_stratified_dfs_gain(df_A2_w_firstcollabs_only)

In [44]:
df_rj.MAGAID.nunique(), df_rm.MAGAID.nunique(), df_rs.MAGAID.nunique(), df_nrj.MAGAID.nunique(), df_nrm.MAGAID.nunique(), df_nrs.MAGAID.nunique()

(69, 125, 120, 69, 125, 120)

In [45]:
# Let us extract the mean dataframes and merge them for different age categories

def get_mean_df_gain(dfr, dfnr):
    mean_dfr = dfr.groupby('MAGAID')[['CollabAcademicAgeAtCollaboration',
                      'CollabMAGCumPapersAtCollaboration',
                      'CollabMAGCumCitationsAtCollaboration',
                      'CollabMAGCumCollaboratorsAtCollaboration']].mean()
    
    mean_dfnr = dfnr.groupby(['MAGAID','MatchMAGAID'])[['CollabAcademicAgeAtCollaboration',
                      'CollabMAGCumPapersAtCollaboration',
                      'CollabMAGCumCitationsAtCollaboration',
                      'CollabMAGCumCollaboratorsAtCollaboration']].mean()\
                    .groupby('MAGAID').mean()\
                    .rename(columns={'CollabMAGCumPapersAtCollaboration':'MatchCollabMAGCumPapersAtCollaboration',
                                    'CollabAcademicAgeAtCollaboration':'MatchCollabAcademicAgeAtCollaboration',
                                    'CollabMAGCumCitationsAtCollaboration': 'MatchCollabMAGCumCitationsAtCollaboration',
                                    'CollabMAGCumCollaboratorsAtCollaboration': 'MatchCollabMAGCumCollaboratorsAtCollaboration'})
    
    return mean_dfr, mean_dfnr

def mean_confidence_interval(data, confidence=0.95):
    a = 1.0 * np.array(data)
    n = len(a)
    m, se = np.mean(a), stats.sem(a)
    h = se * stats.t.ppf((1 + confidence) / 2., n-1)
    return m, m-h, m+h

def get_stats(dfr, dfnr, column):
    """
    This code will compute the mean, median, std dev. and p-value (as per welch test), and CIs for
    the given column
    """
    
    mean_r = dfr[column].mean()
    median_r = dfr[column].median()
    std_r = dfr[column].std()
    
    mean_nr = dfnr['Match'+column].mean()
    median_nr = dfnr['Match'+column].median()
    std_nr = dfnr['Match'+column].std()
    
    _, pval = stats.ttest_ind(dfr[column],dfnr['Match'+column], equal_var=False)
    
    lst_delta = (dfr[column]- dfnr['Match'+column]).tolist()
    delta_mean, conf_lower, conf_upper = mean_confidence_interval(lst_delta, confidence=0.95)
    
    return {column+'_retracted_mean':round(mean_r,2), 
            column+'_retracted_median':round(median_r,2), 
            column+'_retracted_std':round(std_r,2), 
            column+'_nonretracted_mean':round(mean_nr,2), 
            column+'_nonretracted_median':round(median_nr,2), 
            column+'_nonretracted_std':round(std_nr,2), 
            column+'_delta_mean':round(delta_mean,2), 
            column+'_pval_welch':round(pval,3), 
            column+'_CI_95lower':round(conf_lower,2), 
            column+'_CI_95upper':round(conf_upper,2)}

In [46]:
# Now let us do the comparison

# Let us first get the mean dataframes

mean_dfrj, mean_dfnrj = get_mean_df_gain(df_rj, df_nrj)
mean_dfrm, mean_dfnrm = get_mean_df_gain(df_rm, df_nrm)
mean_dfrs, mean_dfnrs = get_mean_df_gain(df_rs, df_nrs)

mean_dfnrj

,MatchCollabAcademicAgeAtCollaboration,MatchCollabMAGCumPapersAtCollaboration,MatchCollabMAGCumCitationsAtCollaboration,MatchCollabMAGCumCollaboratorsAtCollaboration
MAGAID,,,,
48740240,9.166667,34.000000,624.500000,71.333333
115663519,3.840000,13.906667,183.520000,52.933333
207280435,12.034483,37.310345,563.172414,90.241379
409654858,11.400000,40.246154,1196.446154,115.800000
1126006285,13.444444,28.666667,481.666667,57.333333
...,...,...,...,...
2889150962,7.543082,22.223316,528.895626,57.686717
2890048425,5.318182,45.909091,921.818182,122.590909
2935995536,0.321429,1.964286,0.892857,4.892857


In [47]:
exp_fields = ['CollabAcademicAgeAtCollaboration',
                      'CollabMAGCumPapersAtCollaboration',
                      'CollabMAGCumCitationsAtCollaboration',
                      'CollabMAGCumCollaboratorsAtCollaboration']

# Now we should compute outcome variabels for each of the four experience variables.

lst_dicts_gain = []

for exp_field in exp_fields:
    dicts_gain = {}
    
    dict_stats_j = get_stats(mean_dfrj, mean_dfnrj, exp_field)
    dict_stats_m = get_stats(mean_dfrm, mean_dfnrm, exp_field)
    dict_stats_s = get_stats(mean_dfrs, mean_dfnrs, exp_field)
    
    dicts_gain['Misconduct'] = dict_stats_j
    dicts_gain['Plagiarism'] = dict_stats_m
    dicts_gain['Mistake'] = dict_stats_s
    
    lst_dicts_gain.append(dicts_gain)

In [48]:
pd.DataFrame(lst_dicts_gain[0])

,Misconduct,Plagiarism,Mistake
CollabAcademicAgeAtCollaboration_retracted_mean,7.220,6.490,8.890
CollabAcademicAgeAtCollaboration_retracted_median,6.500,6.100,8.670
CollabAcademicAgeAtCollaboration_retracted_std,6.340,4.830,4.970
CollabAcademicAgeAtCollaboration_nonretracted_mean,7.540,7.730,7.950
CollabAcademicAgeAtCollaboration_nonretracted_median,7.400,7.330,7.820
CollabAcademicAgeAtCollaboration_nonretracted_std,3.970,5.210,3.950
CollabAcademicAgeAtCollaboration_delta_mean,-0.320,-1.240,0.940
CollabAcademicAgeAtCollaboration_pval_welch,0.722,0.052,0.107
CollabAcademicAgeAtCollaboration_CI_95lower,-1.810,-2.410,-0.110
CollabAcademicAgeAtCollaboration_CI_95upper,1.170,-0.070,1.980


In [49]:
pd.DataFrame(lst_dicts_gain[1])

,Misconduct,Plagiarism,Mistake
CollabMAGCumPapersAtCollaboration_retracted_mean,40.070,29.840,43.200
CollabMAGCumPapersAtCollaboration_retracted_median,28.000,20.920,35.890
CollabMAGCumPapersAtCollaboration_retracted_std,42.090,32.650,35.950
CollabMAGCumPapersAtCollaboration_nonretracted_mean,35.180,31.830,37.480
CollabMAGCumPapersAtCollaboration_nonretracted_median,32.470,27.890,29.130
CollabMAGCumPapersAtCollaboration_nonretracted_std,21.420,24.460,32.200
CollabMAGCumPapersAtCollaboration_delta_mean,4.880,-1.990,5.720
CollabMAGCumPapersAtCollaboration_pval_welch,0.392,0.587,0.195
CollabMAGCumPapersAtCollaboration_CI_95lower,-5.800,-8.830,-2.420
CollabMAGCumPapersAtCollaboration_CI_95upper,15.560,4.850,13.870


In [50]:
pd.DataFrame(lst_dicts_gain[2])

,Misconduct,Plagiarism,Mistake
CollabMAGCumCitationsAtCollaboration_retracted_mean,1076.050,493.040,1044.050
CollabMAGCumCitationsAtCollaboration_retracted_median,377.000,208.300,626.400
CollabMAGCumCitationsAtCollaboration_retracted_std,2407.450,807.300,1289.480
CollabMAGCumCitationsAtCollaboration_nonretracted_mean,701.720,569.000,918.020
CollabMAGCumCitationsAtCollaboration_nonretracted_median,455.600,299.030,398.870
CollabMAGCumCitationsAtCollaboration_nonretracted_std,775.030,740.290,1590.280
CollabMAGCumCitationsAtCollaboration_delta_mean,374.320,-75.960,126.040
CollabMAGCumCitationsAtCollaboration_pval_welch,0.222,0.439,0.501
CollabMAGCumCitationsAtCollaboration_CI_95lower,-219.000,-249.690,-239.470
CollabMAGCumCitationsAtCollaboration_CI_95upper,967.650,97.770,491.540


In [51]:
pd.DataFrame(lst_dicts_gain[3])

,Misconduct,Plagiarism,Mistake
CollabMAGCumCollaboratorsAtCollaboration_retracted_mean,121.880,98.820,181.710
CollabMAGCumCollaboratorsAtCollaboration_retracted_median,74.430,39.760,74.440
CollabMAGCumCollaboratorsAtCollaboration_retracted_std,157.370,179.720,317.590
CollabMAGCumCollaboratorsAtCollaboration_nonretracted_mean,115.580,85.220,108.250
CollabMAGCumCollaboratorsAtCollaboration_nonretracted_median,75.520,55.550,71.710
CollabMAGCumCollaboratorsAtCollaboration_nonretracted_std,155.200,89.080,117.650
CollabMAGCumCollaboratorsAtCollaboration_delta_mean,6.310,13.600,73.460
CollabMAGCumCollaboratorsAtCollaboration_pval_welch,0.813,0.449,0.019
CollabMAGCumCollaboratorsAtCollaboration_CI_95lower,-48.730,-18.110,14.620
CollabMAGCumCollaboratorsAtCollaboration_CI_95upper,61.340,45.310,132.300


In [52]:
def create_latex_for_filling(dicto, col):
    
    def create_string(metric):
        string = ""
        if metric == 'pval_welch':
            string = "& " + \
                str(dicto.get('Misconduct').get(col+"_"+metric)) + \
                " & " + \
                str(dicto.get('Misconduct').get(col+"_"+metric)) + \
                " & " + \
                str(dicto.get('Plagiarism').get(col+"_"+metric)) + \
                " & " + \
                str(dicto.get('Plagiarism').get(col+"_"+metric)) + \
                " & " + \
                str(dicto.get('Mistake').get(col+"_"+metric)) + \
                " & " + \
                str(dicto.get('Mistake').get(col+"_"+metric)) + \
                "\\\ \n"
        else:
            string = "& " + \
                    str(dicto.get('Misconduct').get(col+"_retracted_"+metric)) + \
                    " & " + \
                    str(dicto.get('Misconduct').get(col+"_nonretracted_"+metric)) + \
                    " & " + \
                    str(dicto.get('Plagiarism').get(col+"_retracted_"+metric)) + \
                    " & " + \
                    str(dicto.get('Plagiarism').get(col+"_nonretracted_"+metric)) + \
                    " & " + \
                    str(dicto.get('Mistake').get(col+"_retracted_"+metric)) + \
                    " & " + \
                    str(dicto.get('Mistake').get(col+"_nonretracted_"+metric)) + \
                    "\\\ \n"
        
        
        
        return string
    
    print(create_string("mean"))
    print(create_string("median"))
    print(create_string("std"))
    print(create_string("pval_welch"))
    

for i in range(len(lst_dicts_retention)):
    dicto = lst_dicts_retention[i]
    col = exp_fields[i]
    print(col)
    create_latex_for_filling(dicto, col)
    
    
# pd.DataFrame(lst_dicts_retention[0])

CollabAcademicAgeAtCollaboration
& 12.92 & 14.68 & 12.61 & 15.41 & 15.11 & 15.88\\ 

& 13.86 & 13.36 & 12.5 & 13.73 & 15.46 & 14.64\\ 

& 7.12 & 6.8 & 6.97 & 7.87 & 6.75 & 7.14\\ 

& 0.154 & 0.154 & 0.003 & 0.003 & 0.401 & 0.401\\ 

CollabMAGCumPapersAtCollaboration
& 69.8 & 93.75 & 63.89 & 66.41 & 69.93 & 74.76\\ 

& 46.51 & 63.28 & 44.0 & 57.0 & 53.0 & 61.67\\ 

& 96.54 & 176.22 & 68.83 & 59.84 & 72.68 & 55.39\\ 

& 0.343 & 0.343 & 0.759 & 0.759 & 0.572 & 0.572\\ 

CollabMAGCumCitationsAtCollaboration
& 1498.48 & 1683.14 & 1158.96 & 1009.08 & 1487.99 & 1568.09\\ 

& 532.42 & 1037.17 & 410.5 & 497.78 & 862.22 & 924.0\\ 

& 2443.6 & 2759.87 & 2846.24 & 1440.51 & 1947.44 & 2243.95\\ 

& 0.689 & 0.689 & 0.603 & 0.603 & 0.773 & 0.773\\ 

CollabMAGCumCollaboratorsAtCollaboration
& 194.34 & 272.99 & 140.04 & 140.57 & 175.55 & 162.51\\ 

& 102.0 & 142.93 & 70.86 & 95.25 & 101.0 & 120.29\\ 

& 326.47 & 712.28 & 175.55 & 170.37 & 203.2 & 149.9\\ 

& 0.424 & 0.424 & 0.981 & 0.981 & 0.58 & 0.58\

In [53]:
for i in range(len(lst_dicts_gain)):
    dicto = lst_dicts_gain[i]
    col = exp_fields[i]
    print(col)
    create_latex_for_filling(dicto, col)

CollabAcademicAgeAtCollaboration
& 7.22 & 7.54 & 6.49 & 7.73 & 8.89 & 7.95\\ 

& 6.5 & 7.4 & 6.1 & 7.33 & 8.67 & 7.82\\ 

& 6.34 & 3.97 & 4.83 & 5.21 & 4.97 & 3.95\\ 

& 0.722 & 0.722 & 0.052 & 0.052 & 0.107 & 0.107\\ 

CollabMAGCumPapersAtCollaboration
& 40.07 & 35.18 & 29.84 & 31.83 & 43.2 & 37.48\\ 

& 28.0 & 32.47 & 20.92 & 27.89 & 35.89 & 29.13\\ 

& 42.09 & 21.42 & 32.65 & 24.46 & 35.95 & 32.2\\ 

& 0.392 & 0.392 & 0.587 & 0.587 & 0.195 & 0.195\\ 

CollabMAGCumCitationsAtCollaboration
& 1076.05 & 701.72 & 493.04 & 569.0 & 1044.05 & 918.02\\ 

& 377.0 & 455.6 & 208.3 & 299.03 & 626.4 & 398.87\\ 

& 2407.45 & 775.03 & 807.3 & 740.29 & 1289.48 & 1590.28\\ 

& 0.222 & 0.222 & 0.439 & 0.439 & 0.501 & 0.501\\ 

CollabMAGCumCollaboratorsAtCollaboration
& 121.88 & 115.58 & 98.82 & 85.22 & 181.71 & 108.25\\ 

& 74.43 & 75.52 & 39.76 & 55.55 & 74.44 & 71.71\\ 

& 157.37 & 155.2 & 179.72 & 89.08 & 317.59 & 117.65\\ 

& 0.813 & 0.813 & 0.449 & 0.449 & 0.019 & 0.019\\ 



### A3: Collaborators retained vs lost: retracted vs. matched

In [54]:
#Let us now modify df_A3 such that we remove all rows with collaborations pre-retraction

df_A3_post = df_A[df_A['PrePostFlag5']=='post5']
df_A3_pre = df_A[df_A['PrePostFlag5']=='pre']

In [55]:
# Now we shall groupby MAGAID, MAGCollabAID, RetractionYear, and sort by MAGCollaborationYear
# Then I shall extract the earliest collaboration year post retraction

df_A3_firstcollabs = df_A3_post.groupby(['MAGAID','MAGCollabAID','RetractionYear'])['MAGCollaborationYear']\
                        .min().reset_index()\
                        .rename(columns={'MAGCollaborationYear':'FirstPostRetractionMAGCollaborationYear'})


# Now we shall merge the new column with A1

df_A3_w_firstcollabs = df_A3_post.merge(df_A1_firstcollabs,
                                   on=['MAGAID','MAGCollabAID','RetractionYear'])

df_A3_w_firstcollabs.shape


(66141, 36)

In [56]:
# Sensibility checks

df_A3_w_firstcollabs.sort_values(by=['MAGAID','MAGCollabAID','MAGCollaborationYear'])\
            [['MAGAID','MAGCollabAID','MAGCollaborationYear','FirstPostRetractionMAGCollaborationYear']].head(30)

,MAGAID,MAGCollabAID,MAGCollaborationYear,FirstPostRetractionMAGCollaborationYear
2121,19100288.0,18011520,2004.0,2004.0
2128,19100288.0,121410733,2004.0,2004.0
2109,19100288.0,410625722,2005.0,2005.0
2122,19100288.0,1235268530,2004.0,2004.0
2131,19100288.0,1340583028,2006.0,2006.0
2103,19100288.0,1793107545,2003.0,2003.0
2104,19100288.0,1793107545,2004.0,2003.0
2123,19100288.0,1859744180,2004.0,2004.0
2105,19100288.0,1863203661,2003.0,2003.0
2106,19100288.0,1863203661,2004.0,2003.0


In [57]:
# Now let us only extract rows where collaboration year is the first collaboration year

df_A3_w_firstcollabs_only = df_A3_w_firstcollabs[df_A3_w_firstcollabs.MAGCollaborationYear == \
                                                df_A3_w_firstcollabs.FirstPostRetractionMAGCollaborationYear]

df_A3_w_firstcollabs_only.shape

(50527, 36)

In [58]:
df_A3_w_firstcollabs_only.columns

Index(['MAGAID', 'MAGCollabAID', 'RetractionYear', 'MAGCollaborationYear',
       'ScientistType', 'CollabGenderizeGender', 'CollabGenderizeConfidence',
       'CollabMAGFirstPubYear', 'CollabMAGCumPapersYearAtRetraction',
       'CollabMAGCumPapersAtRetraction',
       'CollabMAGCumCitationsYearAtRetraction',
       'CollabMAGCumCitationsAtRetraction',
       'CollabMAGCumCollaboratorsYearAtRetraction',
       'CollabMAGCumCollaboratorsAtRetraction',
       'CollabMAGCumPapersYearAtCollaboration',
       'CollabMAGCumPapersAtCollaboration',
       'CollabMAGCumCitationsYearAtCollaboration',
       'CollabMAGCumCitationsAtCollaboration',
       'CollabMAGCumCollaboratorsYearAtCollaboration',
       'CollabMAGCumCollaboratorsAtCollaboration',
       'ReasonPropagatedMajorityOfMajority', 'MAGAIDFirstORLastAuthorFlag',
       'CollabAcademicAgeAtRetraction', 'CollabAcademicAgeAtCollaboration',
       'PrePostFlag5', 'post5', 'pre', 'NumRetentionW5', 'CollabAIDRetainedW5',
       'CollabAI

In [59]:
# Finally let us merge post and pre

df_A3_post_pre = pd.concat([df_A3_w_firstcollabs_only,df_A3_pre])

df_A3_post_pre.head()

,MAGAID,MAGCollabAID,RetractionYear,MAGCollaborationYear,ScientistType,CollabGenderizeGender,CollabGenderizeConfidence,CollabMAGFirstPubYear,CollabMAGCumPapersYearAtRetraction,CollabMAGCumPapersAtRetraction,...,pre,NumRetentionW5,CollabAIDRetainedW5,CollabAIDLostW5,NumNewCollaboratorsW5,CollabAIDGainedW5,CollabAIDinRetained,CollabAIDinGained,CollabAIDinLost,FirstPostRetractionMAGCollaborationYear
2,2.033335e+09,1917877966,1995.0,1998.0,retracted,male,0.99,1994.0,1994.0,1.0,...,"{2041262087, 2590561305, 1940968476, 211428099...",23,"{2690057605, 2041262087, 2466141063, 230478619...","{2590561305, 1940968476, 2657016352, 265692164...",60,"{1973921793, 2308182914, 1147890307, 265602649...",False,True,False,1998.0
4,2.033335e+09,2169118091,1995.0,1998.0,retracted,male,1.00,1995.0,1995.0,1.0,...,"{2041262087, 2590561305, 1940968476, 211428099...",23,"{2690057605, 2041262087, 2466141063, 230478619...","{2590561305, 1940968476, 2657016352, 265692164...",60,"{1973921793, 2308182914, 1147890307, 265602649...",False,True,False,1998.0
7,2.033335e+09,275085591,1995.0,1996.0,retracted,male,0.99,1994.0,1995.0,5.0,...,"{2041262087, 2590561305, 1940968476, 211428099...",23,"{2690057605, 2041262087, 2466141063, 230478619...","{2590561305, 1940968476, 2657016352, 265692164...",60,"{1973921793, 2308182914, 1147890307, 265602649...",True,False,False,1996.0
9,2.033335e+09,2111014462,1995.0,1996.0,retracted,female,0.98,1988.0,1995.0,17.0,...,"{2041262087, 2590561305, 1940968476, 211428099...",23,"{2690057605, 2041262087, 2466141063, 230478619...","{2590561305, 1940968476, 2657016352, 265692164...",60,"{1973921793, 2308182914, 1147890307, 265602649...",True,False,False,1996.0
12,2.033335e+09,2622920657,1995.0,1996.0,retracted,male,0.99,1991.0,1995.0,17.0,...,"{2041262087, 2590561305, 1940968476, 211428099...",23,"{2690057605, 2041262087, 2466141063, 230478619...","{2590561305, 1940968476, 2657016352, 265692164...",60,"{1973921793, 2308182914, 1147890307, 265602649...",True,False,False,1996.0


In [60]:
def create_stratified_dfs_a3(dfi):
    
    # This function will create 6 dataframes relevant for conducting our analysis
    # 3 of those dataframes will be for relevant columns for treatment
    # rest 3 will be average control. 
    # These will be stratified by treatment and control, and further stratified by seniority
    df_ids = pd.read_csv(INDIR_MATCHING+"/RWMAG_rematched_control_augmented_rematching_30perc.csv",
                    usecols=['MAGAID','MatchMAGAID', 'RetractionYear']).drop_duplicates()
    
    rel_cols = ['MAGAID', 'ScientistType','MAGCollabAID', 'RetractionYear',
               'CollabMAGCumPapersAtRetraction', 'CollabMAGCumCitationsAtRetraction',
               'CollabMAGCumCollaboratorsAtRetraction', 'ReasonPropagatedMajorityOfMajority',
               'CollabAcademicAgeAtRetraction', 'CollabAIDinRetained', 'CollabAIDinLost']
    
    # Only extracting relevant cols
    dfi = dfi[rel_cols].drop_duplicates()
    
    # Only extract those collaborators that were retained
    dfi_retained = dfi[dfi['CollabAIDinRetained']]
    dfi_lost = dfi[dfi['CollabAIDinLost']]
    
    # Dividing into retracted and matched
    df_retracted_retained = dfi_retained[dfi_retained.ScientistType == 'retracted']
    df_retracted_lost = dfi_lost[dfi_lost.ScientistType == 'retracted']
    
    df_nonretracted_retained = dfi_retained[dfi_retained.ScientistType == 'matched']\
                        .rename(columns={'MAGAID':'MatchMAGAID'})\
                        .merge(df_ids, on=['MatchMAGAID','RetractionYear'])
    
    df_nonretracted_lost = dfi_lost[dfi_lost.ScientistType == 'matched']\
                        .rename(columns={'MAGAID':'MatchMAGAID'})\
                        .merge(df_ids, on=['MatchMAGAID','RetractionYear'])
    
    # We also need to makre sure that the four groups retracted,non-retracted,retained,lost have same ids
    
    set1 = set(df_retracted_retained['MAGAID'].unique())
    set2 = set(df_retracted_lost['MAGAID'].unique())
    set3 = set(df_nonretracted_retained['MAGAID'].unique())
    set4 = set(df_nonretracted_lost['MAGAID'].unique())
    
    magaids_intersection = set1.intersection(set2, set3, set4)
    
    df_retracted_retained = df_retracted_retained[df_retracted_retained.MAGAID.isin(magaids_intersection)]
    df_retracted_lost = df_retracted_lost[df_retracted_lost.MAGAID.isin(magaids_intersection)]
    df_nonretracted_retained = df_nonretracted_retained[df_nonretracted_retained.MAGAID.isin(magaids_intersection)]
    df_nonretracted_lost = df_nonretracted_lost[df_nonretracted_lost.MAGAID.isin(magaids_intersection)]

    
    # Dividing into seniority for retracted retained
    
    dfrj_r = df_retracted_retained[df_retracted_retained.ReasonPropagatedMajorityOfMajority=='misconduct']
    dfrm_r = df_retracted_retained[df_retracted_retained.ReasonPropagatedMajorityOfMajority=='plagiarism']
    dfrs_r = df_retracted_retained[df_retracted_retained.ReasonPropagatedMajorityOfMajority=='mistake']
    
    dfrj_l = df_retracted_lost[df_retracted_lost.ReasonPropagatedMajorityOfMajority=='misconduct']
    dfrm_l = df_retracted_lost[df_retracted_lost.ReasonPropagatedMajorityOfMajority=='plagiarism']
    dfrs_l = df_retracted_lost[df_retracted_lost.ReasonPropagatedMajorityOfMajority=='mistake']
    
    # and matched
    dfnrj_r = df_nonretracted_retained[df_nonretracted_retained.ReasonPropagatedMajorityOfMajority=='misconduct']
    dfnrm_r = df_nonretracted_retained[df_nonretracted_retained.ReasonPropagatedMajorityOfMajority=='plagiarism']
    dfnrs_r = df_nonretracted_retained[df_nonretracted_retained.ReasonPropagatedMajorityOfMajority=='mistake']
    
    dfnrj_l = df_nonretracted_lost[df_nonretracted_lost.ReasonPropagatedMajorityOfMajority=='misconduct']
    dfnrm_l = df_nonretracted_lost[df_nonretracted_lost.ReasonPropagatedMajorityOfMajority=='plagiarism']
    dfnrs_l = df_nonretracted_lost[df_nonretracted_lost.ReasonPropagatedMajorityOfMajority=='mistake']
    
    return [dfrj_r,dfrm_r,dfrs_r,dfrj_l,dfrm_l,dfrs_l,dfnrj_r,dfnrm_r,dfnrs_r,dfnrj_l,dfnrm_l,dfnrs_l]
    

In [61]:
lst_stratified_dfs = create_stratified_dfs_a3(df_A3_post_pre)

for dfj in lst_stratified_dfs:
    print(dfj.MAGAID.nunique())
    
dfrj_r,dfrm_r,dfrs_r,dfrj_l,dfrm_l,dfrs_l,dfnrj_r,dfnrm_r,dfnrs_r,dfnrj_l,dfnrm_l,dfnrs_l = lst_stratified_dfs

62
115
111
62
115
111
62
115
111
62
115
111


In [62]:
# Let us extract the mean dataframes and merge them for different age categories

def get_mean_df_a3(dfr, dfnr):
    mean_dfr = dfr.groupby('MAGAID')[['CollabAcademicAgeAtRetraction',
                      'CollabMAGCumPapersAtRetraction',
                      'CollabMAGCumCitationsAtRetraction',
                      'CollabMAGCumCollaboratorsAtRetraction']].mean()
    
    mean_dfnr = dfnr.groupby(['MAGAID','MatchMAGAID'])[['CollabAcademicAgeAtRetraction',
                      'CollabMAGCumPapersAtRetraction',
                      'CollabMAGCumCitationsAtRetraction',
                      'CollabMAGCumCollaboratorsAtRetraction']].mean()\
                    .groupby('MAGAID').mean()\
                    .rename(columns={'CollabMAGCumPapersAtRetraction':'MatchCollabMAGCumPapersAtRetraction',
                                    'CollabAcademicAgeAtRetraction':'MatchCollabAcademicAgeAtRetraction',
                                    'CollabMAGCumCitationsAtRetraction': 'MatchCollabMAGCumCitationsAtRetraction',
                                    'CollabMAGCumCollaboratorsAtRetraction': 'MatchCollabMAGCumCollaboratorsAtRetraction'})
    
    return mean_dfr, mean_dfnr

def mean_confidence_interval(data, confidence=0.95):
    a = 1.0 * np.array(data)
    n = len(a)
    m, se = np.mean(a), stats.sem(a)
    h = se * stats.t.ppf((1 + confidence) / 2., n-1)
    return m, m-h, m+h

def get_stats(dfr, dfnr, column):
    """
    This code will compute the mean, median, std dev. and p-value (as per welch test), and CIs for
    the given column
    """
    
    mean_r = dfr[column].mean()
    median_r = dfr[column].median()
    std_r = dfr[column].std()
    
    mean_nr = dfnr['Match'+column].mean()
    median_nr = dfnr['Match'+column].median()
    std_nr = dfnr['Match'+column].std()
    
    _, pval = stats.ttest_ind(dfr[column],dfnr['Match'+column], equal_var=False)
    
    lst_delta = (dfr[column]- dfnr['Match'+column]).tolist()
    delta_mean, conf_lower, conf_upper = mean_confidence_interval(lst_delta, confidence=0.95)
    
    return {column+'_retracted_mean':round(mean_r,2), 
            column+'_retracted_median':round(median_r,2), 
            column+'_retracted_std':round(std_r,2), 
            column+'_nonretracted_mean':round(mean_nr,2), 
            column+'_nonretracted_median':round(median_nr,2), 
            column+'_nonretracted_std':round(std_nr,2), 
            column+'_delta_mean':round(delta_mean,2), 
            column+'_pval_welch':round(pval,3), 
            column+'_CI_95lower':round(conf_lower,2), 
            column+'_CI_95upper':round(conf_upper,2)}

In [63]:
# Now let us do the comparison

# Let us first get the mean dataframes

mean_dfrj_r, mean_dfnrj_r = get_mean_df_a3(dfrj_r, dfnrj_r)
mean_dfrj_l, mean_dfnrj_l = get_mean_df_a3(dfrj_l, dfnrj_l)

mean_dfrm_r, mean_dfnrm_r = get_mean_df_a3(dfrm_r, dfnrm_r)
mean_dfrm_l, mean_dfnrm_l = get_mean_df_a3(dfrm_l, dfnrm_l)

mean_dfrs_r, mean_dfnrs_r = get_mean_df_a3(dfrs_r, dfnrs_r)
mean_dfrs_l, mean_dfnrs_l = get_mean_df_a3(dfrs_l, dfnrs_l)


# Now let us compute differences

def compute_diff_df(df_ri, df_li, scientistType='retracted'):
    
    dfrli = df_ri.merge(df_li, right_index=True, left_index=True)
    
    if scientistType == 'matched':
        
        dfrli['MatchDiffAcademicAgeAtRetraction'] = dfrli['MatchCollabAcademicAgeAtRetraction_x'] - \
                                                dfrli['MatchCollabAcademicAgeAtRetraction_y']
        
        dfrli['MatchDiffMAGCumPapersAtRetraction'] = dfrli['MatchCollabMAGCumPapersAtRetraction_x'] - \
                                                dfrli['MatchCollabMAGCumPapersAtRetraction_y']
        
        dfrli['MatchDiffMAGCumCitationsAtRetraction'] = dfrli['MatchCollabMAGCumCitationsAtRetraction_x'] - \
                                                dfrli['MatchCollabMAGCumCitationsAtRetraction_y']
        
        dfrli['MatchDiffMAGCumCollaboratorsAtRetraction'] = dfrli['MatchCollabMAGCumCollaboratorsAtRetraction_x'] - \
                                                dfrli['MatchCollabMAGCumCollaboratorsAtRetraction_y']
        
        return dfrli
    
        
    dfrli['DiffAcademicAgeAtRetraction'] = dfrli['CollabAcademicAgeAtRetraction_x'] - \
                                            dfrli['CollabAcademicAgeAtRetraction_y']

    dfrli['DiffMAGCumPapersAtRetraction'] = dfrli['CollabMAGCumPapersAtRetraction_x'] - \
                                            dfrli['CollabMAGCumPapersAtRetraction_y']

    dfrli['DiffMAGCumCitationsAtRetraction'] = dfrli['CollabMAGCumCitationsAtRetraction_x'] - \
                                            dfrli['CollabMAGCumCitationsAtRetraction_y']

    dfrli['DiffMAGCumCollaboratorsAtRetraction'] = dfrli['CollabMAGCumCollaboratorsAtRetraction_x'] - \
                                            dfrli['CollabMAGCumCollaboratorsAtRetraction_y']

    return dfrli
    


In [64]:
dfrj_rMinusl = compute_diff_df(mean_dfrj_r, mean_dfrj_l)
dfnrj_rMinusl = compute_diff_df(mean_dfnrj_r, mean_dfnrj_l, scientistType='matched')


dfrm_rMinusl = compute_diff_df(mean_dfrm_r, mean_dfrm_l)
dfnrm_rMinusl = compute_diff_df(mean_dfnrm_r, mean_dfnrm_l, scientistType='matched')

dfrs_rMinusl = compute_diff_df(mean_dfrs_r, mean_dfrs_l)
dfnrs_rMinusl = compute_diff_df(mean_dfnrs_r, mean_dfnrs_l, scientistType='matched')

In [65]:
exp_fields = ['DiffAcademicAgeAtRetraction',
              'DiffMAGCumPapersAtRetraction',
              'DiffMAGCumCitationsAtRetraction',
              'DiffMAGCumCollaboratorsAtRetraction']

# Now we should compute outcome variabels for each of the four experience variables.

lst_dicts_a3 = []

for exp_field in exp_fields:
    dicts_a3 = {}
    
    dict_stats_j = get_stats(dfrj_rMinusl, dfnrj_rMinusl, exp_field)
    dict_stats_m = get_stats(dfrm_rMinusl, dfnrm_rMinusl, exp_field)
    dict_stats_s = get_stats(dfrs_rMinusl, dfnrs_rMinusl, exp_field)
    
    dicts_a3['Misconduct'] = dict_stats_j
    dicts_a3['Plagiarism'] = dict_stats_m
    dicts_a3['Mistake'] = dict_stats_s
    
    lst_dicts_a3.append(dicts_a3)

In [66]:
pd.DataFrame(lst_dicts_a3[0])

,Misconduct,Plagiarism,Mistake
DiffAcademicAgeAtRetraction_retracted_mean,0.850,1.59,2.590
DiffAcademicAgeAtRetraction_retracted_median,0.150,1.00,2.260
DiffAcademicAgeAtRetraction_retracted_std,6.640,6.08,6.300
DiffAcademicAgeAtRetraction_nonretracted_mean,1.890,3.07,2.520
DiffAcademicAgeAtRetraction_nonretracted_median,1.010,1.62,1.740
DiffAcademicAgeAtRetraction_nonretracted_std,6.900,7.79,6.470
DiffAcademicAgeAtRetraction_delta_mean,-1.030,-1.48,0.070
DiffAcademicAgeAtRetraction_pval_welch,0.398,0.11,0.938
DiffAcademicAgeAtRetraction_CI_95lower,-3.180,-3.09,-1.460
DiffAcademicAgeAtRetraction_CI_95upper,1.120,0.13,1.590


In [67]:
pd.DataFrame(lst_dicts_a3[1])

,Misconduct,Plagiarism,Mistake
DiffMAGCumPapersAtRetraction_retracted_mean,18.570,21.280,19.980
DiffMAGCumPapersAtRetraction_retracted_median,5.290,10.760,11.360
DiffMAGCumPapersAtRetraction_retracted_std,81.920,53.050,60.350
DiffMAGCumPapersAtRetraction_nonretracted_mean,37.570,23.670,23.290
DiffMAGCumPapersAtRetraction_nonretracted_median,13.990,13.080,11.890
DiffMAGCumPapersAtRetraction_nonretracted_std,136.100,57.890,49.720
DiffMAGCumPapersAtRetraction_delta_mean,-19.000,-2.400,-3.310
DiffMAGCumPapersAtRetraction_pval_welch,0.349,0.744,0.656
DiffMAGCumPapersAtRetraction_CI_95lower,-59.840,-17.220,-18.700
DiffMAGCumPapersAtRetraction_CI_95upper,21.840,12.430,12.090


In [68]:
pd.DataFrame(lst_dicts_a3[2])

,Misconduct,Plagiarism,Mistake
DiffMAGCumCitationsAtRetraction_retracted_mean,267.930,264.130,81.250
DiffMAGCumCitationsAtRetraction_retracted_median,3.820,-0.100,59.010
DiffMAGCumCitationsAtRetraction_retracted_std,1695.690,1459.280,1554.050
DiffMAGCumCitationsAtRetraction_nonretracted_mean,481.130,217.330,344.660
DiffMAGCumCitationsAtRetraction_nonretracted_median,94.670,24.000,111.300
DiffMAGCumCitationsAtRetraction_nonretracted_std,1794.340,1175.920,1603.380
DiffMAGCumCitationsAtRetraction_delta_mean,-213.200,46.800,-263.410
DiffMAGCumCitationsAtRetraction_pval_welch,0.498,0.789,0.215
DiffMAGCumCitationsAtRetraction_CI_95lower,-854.340,-296.920,-711.180
DiffMAGCumCitationsAtRetraction_CI_95upper,427.950,390.520,184.360


In [69]:
pd.DataFrame(lst_dicts_a3[3])

,Misconduct,Plagiarism,Mistake
DiffMAGCumCollaboratorsAtRetraction_retracted_mean,4.480,21.290,32.730
DiffMAGCumCollaboratorsAtRetraction_retracted_median,1.880,8.200,12.500
DiffMAGCumCollaboratorsAtRetraction_retracted_std,329.310,142.850,120.910
DiffMAGCumCollaboratorsAtRetraction_nonretracted_mean,73.670,34.670,31.860
DiffMAGCumCollaboratorsAtRetraction_nonretracted_median,31.940,19.140,15.800
DiffMAGCumCollaboratorsAtRetraction_nonretracted_std,460.630,136.040,125.690
DiffMAGCumCollaboratorsAtRetraction_delta_mean,-69.200,-13.380,0.870
DiffMAGCumCollaboratorsAtRetraction_pval_welch,0.338,0.468,0.958
DiffMAGCumCollaboratorsAtRetraction_CI_95lower,-218.610,-50.950,-32.490
DiffMAGCumCollaboratorsAtRetraction_CI_95upper,80.210,24.190,34.220


In [70]:
def create_latex_for_filling(dicto, col):
    
    def create_string(metric):
        string = ""
        if metric == 'pval_welch':
            string = "& " + \
                str(dicto.get('Misconduct').get(col+"_"+metric)) + \
                " & " + \
                str(dicto.get('Misconduct').get(col+"_"+metric)) + \
                " & " + \
                str(dicto.get('Plagiarism').get(col+"_"+metric)) + \
                " & " + \
                str(dicto.get('Plagiarism').get(col+"_"+metric)) + \
                " & " + \
                str(dicto.get('Mistake').get(col+"_"+metric)) + \
                " & " + \
                str(dicto.get('Mistake').get(col+"_"+metric)) + \
                "\\\ \n"
        else:
            string = "& " + \
                    str(dicto.get('Misconduct').get(col+"_retracted_"+metric)) + \
                    " & " + \
                    str(dicto.get('Misconduct').get(col+"_nonretracted_"+metric)) + \
                    " & " + \
                    str(dicto.get('Plagiarism').get(col+"_retracted_"+metric)) + \
                    " & " + \
                    str(dicto.get('Plagiarism').get(col+"_nonretracted_"+metric)) + \
                    " & " + \
                    str(dicto.get('Mistake').get(col+"_retracted_"+metric)) + \
                    " & " + \
                    str(dicto.get('Mistake').get(col+"_nonretracted_"+metric)) + \
                    "\\\ \n"
        
        
        
        return string
    
    print(create_string("mean"))
    print(create_string("median"))
    print(create_string("std"))
    print(create_string("pval_welch"))
    

for i in range(len(lst_dicts_a3)):
    dicto = lst_dicts_a3[i]
    col = exp_fields[i]
    print(col)
    create_latex_for_filling(dicto, col)

DiffAcademicAgeAtRetraction
& 0.85 & 1.89 & 1.59 & 3.07 & 2.59 & 2.52\\ 

& 0.15 & 1.01 & 1.0 & 1.62 & 2.26 & 1.74\\ 

& 6.64 & 6.9 & 6.08 & 7.79 & 6.3 & 6.47\\ 

& 0.398 & 0.398 & 0.11 & 0.11 & 0.938 & 0.938\\ 

DiffMAGCumPapersAtRetraction
& 18.57 & 37.57 & 21.28 & 23.67 & 19.98 & 23.29\\ 

& 5.29 & 13.99 & 10.76 & 13.08 & 11.36 & 11.89\\ 

& 81.92 & 136.1 & 53.05 & 57.89 & 60.35 & 49.72\\ 

& 0.349 & 0.349 & 0.744 & 0.744 & 0.656 & 0.656\\ 

DiffMAGCumCitationsAtRetraction
& 267.93 & 481.13 & 264.13 & 217.33 & 81.25 & 344.66\\ 

& 3.82 & 94.67 & -0.1 & 24.0 & 59.01 & 111.3\\ 

& 1695.69 & 1794.34 & 1459.28 & 1175.92 & 1554.05 & 1603.38\\ 

& 0.498 & 0.498 & 0.789 & 0.789 & 0.215 & 0.215\\ 

DiffMAGCumCollaboratorsAtRetraction
& 4.48 & 73.67 & 21.29 & 34.67 & 32.73 & 31.86\\ 

& 1.88 & 31.94 & 8.2 & 19.14 & 12.5 & 15.8\\ 

& 329.31 & 460.63 & 142.85 & 136.04 & 120.91 & 125.69\\ 

& 0.338 & 0.338 & 0.468 & 0.468 & 0.958 & 0.958\\ 



# Processing dictionaries for plots

In [71]:
expfield_categories = ['Academic Age','Number of Papers',
                       'Number of Citations', 'Number of Collaborators']

master_dict = {}

master_dict['Retention'] = {}

for i in range(len(expfield_categories)):
    master_dict['Retention'][expfield_categories[i]] = lst_dicts_retention[i]

master_dict['Gain'] = {}

for i in range(len(expfield_categories)):
    master_dict['Gain'][expfield_categories[i]] = lst_dicts_gain[i]
    
master_dict['DiD'] = {}

for i in range(len(expfield_categories)):
    master_dict['DiD'][expfield_categories[i]] = lst_dicts_a3[i]

In [72]:
master_dict.keys()

dict_keys(['Retention', 'Gain', 'DiD'])

In [73]:
master_dict

{'Retention': {'Academic Age': {'Misconduct': {'CollabAcademicAgeAtCollaboration_retracted_mean': 12.92,
    'CollabAcademicAgeAtCollaboration_retracted_median': 13.86,
    'CollabAcademicAgeAtCollaboration_retracted_std': 7.12,
    'CollabAcademicAgeAtCollaboration_nonretracted_mean': 14.68,
    'CollabAcademicAgeAtCollaboration_nonretracted_median': 13.36,
    'CollabAcademicAgeAtCollaboration_nonretracted_std': 6.8,
    'CollabAcademicAgeAtCollaboration_delta_mean': -1.77,
    'CollabAcademicAgeAtCollaboration_pval_welch': 0.154,
    'CollabAcademicAgeAtCollaboration_CI_95lower': -3.97,
    'CollabAcademicAgeAtCollaboration_CI_95upper': 0.44},
   'Plagiarism': {'CollabAcademicAgeAtCollaboration_retracted_mean': 12.61,
    'CollabAcademicAgeAtCollaboration_retracted_median': 12.5,
    'CollabAcademicAgeAtCollaboration_retracted_std': 6.97,
    'CollabAcademicAgeAtCollaboration_nonretracted_mean': 15.41,
    'CollabAcademicAgeAtCollaboration_nonretracted_median': 13.73,
    'CollabAca

In [74]:
def save_dict(dicto, fname):
    import pickle 

    with open(fname, 'wb') as f:
        pickle.dump(dicto, f)
        
def read_dict(fname):
    import pickle
    
    with open(fname, 'rb') as f:
        loaded_dict = pickle.load(f)
        return loaded_dict

In [75]:
OUTDIR = "/Users/sm9654/desktop/NYUAD/nyuad-research/retraction_openalex/retraction_effects_on_academic_careers/data/plot_data/"

save_dict(master_dict, OUTDIR+"/collaborator_chars_byReason_firstlastauthors.pkl")

In [76]:
master_dict

{'Retention': {'Academic Age': {'Misconduct': {'CollabAcademicAgeAtCollaboration_retracted_mean': 12.92,
    'CollabAcademicAgeAtCollaboration_retracted_median': 13.86,
    'CollabAcademicAgeAtCollaboration_retracted_std': 7.12,
    'CollabAcademicAgeAtCollaboration_nonretracted_mean': 14.68,
    'CollabAcademicAgeAtCollaboration_nonretracted_median': 13.36,
    'CollabAcademicAgeAtCollaboration_nonretracted_std': 6.8,
    'CollabAcademicAgeAtCollaboration_delta_mean': -1.77,
    'CollabAcademicAgeAtCollaboration_pval_welch': 0.154,
    'CollabAcademicAgeAtCollaboration_CI_95lower': -3.97,
    'CollabAcademicAgeAtCollaboration_CI_95upper': 0.44},
   'Plagiarism': {'CollabAcademicAgeAtCollaboration_retracted_mean': 12.61,
    'CollabAcademicAgeAtCollaboration_retracted_median': 12.5,
    'CollabAcademicAgeAtCollaboration_retracted_std': 6.97,
    'CollabAcademicAgeAtCollaboration_nonretracted_mean': 15.41,
    'CollabAcademicAgeAtCollaboration_nonretracted_median': 13.73,
    'CollabAca